<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_CNN_hsj_SHAP_detection_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 10.0 MB/s eta 0:00:00


In [2]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Load trainset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [4]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [5]:
hsj_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/24_feature/hsj/hsj_cnn_bin_feature_10000'

In [6]:
hsj_df = pd.read_csv(hsj_path)

In [7]:
hsj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      10000 non-null  float64
 1   Tot Fwd Pkts       10000 non-null  float64
 2   Tot Bwd Pkts       10000 non-null  float64
 3   TotLen Fwd Pkts    10000 non-null  float64
 4   TotLen Bwd Pkts    10000 non-null  float64
 5   Fwd Pkt Len Max    10000 non-null  float64
 6   Fwd Pkt Len Min    10000 non-null  float64
 7   Fwd Pkt Len Mean   10000 non-null  float64
 8   Fwd Pkt Len Std    10000 non-null  float64
 9   Bwd Pkt Len Max    10000 non-null  float64
 10  Bwd Pkt Len Min    10000 non-null  float64
 11  Bwd Pkt Len Mean   10000 non-null  float64
 12  Bwd Pkt Len Std    10000 non-null  float64
 13  Flow Byts/s        10000 non-null  float64
 14  Flow Pkts/s        10000 non-null  float64
 15  Flow IAT Mean      10000 non-null  float64
 16  Flow IAT Std       1000

# Load DL-based IDS model

In [8]:
model_cnn = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_cnn_bin.h5', compile=True)

# SHAP

In [9]:
explainer = shap.KernelExplainer(model_cnn, shap.sample(df, 100, random_state=42))

In [10]:
hsj_100 = hsj_df.head(100)

42m ~ 1g

In [11]:
shap_values = explainer.shap_values(hsj_100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
len(shap_values)

1

In [ ]:
type(shap_values[0])

numpy.ndarray

In [12]:
len(shap_values[0])

100

In [13]:
shap_values[0].shape

(100, 76)

In [14]:
data_df = pd.DataFrame(shap_values[0])

In [15]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.000060,0.000000,0.000000,0.0,0.0,0.000003,0.0,0.000003,0.000000,0.000000,...,0.000000,0.0,0.000003,0.0,0.000001,0.000007,0.000014,0.000012,0.000000,0.000062
1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000003,0.0,0.000000,0.0,0.000011,0.000006,0.000006,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000013,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000009,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.000158,-0.000028,0.000066,0.0,0.0,0.000000,0.0,-0.000035,-0.000038,-0.000028,...,-0.000049,0.0,0.000000,0.0,0.000000,0.000000,-0.000256,0.000115,0.000589,-0.000469


In [16]:
data_df.columns = df.columns
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      100 non-null    float64
 1   Tot Fwd Pkts       100 non-null    float64
 2   Tot Bwd Pkts       100 non-null    float64
 3   TotLen Fwd Pkts    100 non-null    float64
 4   TotLen Bwd Pkts    100 non-null    float64
 5   Fwd Pkt Len Max    100 non-null    float64
 6   Fwd Pkt Len Min    100 non-null    float64
 7   Fwd Pkt Len Mean   100 non-null    float64
 8   Fwd Pkt Len Std    100 non-null    float64
 9   Bwd Pkt Len Max    100 non-null    float64
 10  Bwd Pkt Len Min    100 non-null    float64
 11  Bwd Pkt Len Mean   100 non-null    float64
 12  Bwd Pkt Len Std    100 non-null    float64
 13  Flow Byts/s        100 non-null    float64
 14  Flow Pkts/s        100 non-null    float64
 15  Flow IAT Mean      100 non-null    float64
 16  Flow IAT Std       100 non-

In [17]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/CNN_HSJ/hsj_100_samples_[0]_v2.csv', index=False)

In [18]:
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000060,0.000000,0.000000,0.0,0.0,0.000003,0.0,0.000003,0.000000,0.000000,...,0.000000,0.0,0.000003,0.0,0.000001,0.000007,0.000014,0.000012,0.000000,0.000062
1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000003,0.0,0.000000,0.0,0.000011,0.000006,0.000006,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000013,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000009,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.000158,-0.000028,0.000066,0.0,0.0,0.000000,0.0,-0.000035,-0.000038,-0.000028,...,-0.000049,0.0,0.000000,0.0,0.000000,0.000000,-0.000256,0.000115,0.000589,-0.000469


In [19]:
data_dict = {}
for idx in range(0, 100):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [20]:
print(data_dict)

{'Adv sample 0': ['SYN Flag Cnt', 'Bwd Pkts/s', 'ACK Flag Cnt', 'FIN Flag Cnt', 'Flow Pkts/s', 'Bwd PSH Flags', 'Init Bwd Win Byts', 'Fwd IAT Tot', 'Flow IAT Max', 'Bwd IAT Tot'], 'Adv sample 1': ['Bwd Pkts/s', 'FIN Flag Cnt', 'ACK Flag Cnt', 'Flow Pkts/s', 'Bwd PSH Flags', 'Fwd IAT Tot', 'Init Bwd Win Byts', 'Bwd IAT Tot', 'Flow IAT Max', 'Down/Up Ratio'], 'Adv sample 2': ['Bwd Pkts/s', 'FIN Flag Cnt', 'ACK Flag Cnt', 'Bwd PSH Flags', 'Flow Pkts/s', 'Fwd IAT Tot', 'Flow IAT Max', 'Bwd IAT Tot', 'Init Bwd Win Byts', 'PSH Flag Cnt'], 'Adv sample 3': ['Bwd Pkts/s', 'FIN Flag Cnt', 'ACK Flag Cnt', 'Flow Pkts/s', 'Bwd PSH Flags', 'Fwd IAT Tot', 'Init Bwd Win Byts', 'Bwd IAT Tot', 'Flow IAT Max', 'PSH Flag Cnt'], 'Adv sample 4': ['Down/Up Ratio', 'Bwd Pkts/s', 'Idle Max', 'SYN Flag Cnt', 'Fwd IAT Tot', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd PSH Flags', 'Idle Std', 'Subflow Fwd Byts'], 'Adv sample 5': ['SYN Flag Cnt', 'ACK Flag Cnt', 'FIN Flag Cnt', 'Bwd PSH Flags', 'Fwd IAT Tot', 'Init Bw

In [21]:
len(shap_values[0])

100

In [22]:
len(data_dict)

100

# Detection phase

In [23]:
import os
import pandas as pd

In [40]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/cnn_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,URG Flag Cnt,149
1,Bwd Pkts/s,148
2,FIN Flag Cnt,141
3,Bwd PSH Flags,131
4,SYN Flag Cnt,123
5,PSH Flag Cnt,116
6,ACK Flag Cnt,98
7,Down/Up Ratio,97
8,Flow Pkts/s,86
9,Init Bwd Win Byts,82


## Detection rate (top 24) = 100/100

In [41]:
whitelist = whitelist_df.iloc[:, 0][:24].tolist()
print(whitelist)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max']


In [42]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:100


## Detection rate (top 38) = 99/100

In [43]:
whitelist2 = whitelist_df.iloc[:, 0][:38].tolist()
print(whitelist2)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'Idle Mean', 'Bwd Seg Size Avg', 'Pkt Len Mean', 'Flow IAT Mean', 'Pkt Len Std', 'TotLen Bwd Pkts', 'Bwd Pkt Len Std', 'Bwd Header Len', 'Subflow Bwd Pkts', 'Fwd Act Data Pkts', 'Fwd Pkt Len Min', 'Pkt Len Min']


In [44]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:99
